In [301]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import random

In [302]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [303]:
# build the vocapbulary of charactors mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print (itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [304]:
block_size = 3

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = jnp.array(X, dtype=jnp.int32)
    Y = jnp.array(Y, dtype=jnp.int32)
    print(X.shape, Y.shape)
    return X, Y


random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))


Xtr, Ytr = build_dataset(words[:n1])  # 80%
Xdev, Ydev = build_dataset(words[n1:n2])  # 10%
Xte, Yte = build_dataset(words[n2:])  # 10%


(182625, 3) (182625,)
(22655, 3) (22655,)
(22866, 3) (22866,)


In [305]:
# Set master seed and key from beginning
seed = 1701
key = jax.random.PRNGKey(seed)

# MLP Model
n_embd = 10
n_hidden = 200

# Embedding Matrix
key, C_key = jax.random.split(key)
C = jax.random.normal(C_key, (vocab_size, n_embd), dtype=jnp.float32)

# First layer weights
key, W1_key, b1_key = jax.random.split(key,num=3)
W1 = jax.random.normal(W1_key, (n_embd * block_size, n_hidden), dtype=jnp.float32)/(n_embd * block_size)**0.5
b1 = jax.random.normal(b1_key, (n_hidden, ), dtype=jnp.float32)*0.01

# Second layer weights
key, W2_key, b2_key = jax.random.split(key,num=3)
W2 = jax.random.normal(W2_key, (n_hidden, vocab_size), dtype=jnp.float32)/n_hidden**0.5
b2 = jax.random.normal(b2_key, (vocab_size, ), dtype=jnp.float32)*0.01

#params = [C, W1, b1, W2, b2]
params = dict(C=C, W1=W1, b1=b1, W2=W2, b2=b2)

#print (params)

#print(f'Network parameter size is: {sum(p.size for p in params)}')

print(f'{Xtr.shape = }')
print(f'{C.shape = }')
print(f'{C[Xtr].shape = }')


Xtr.shape = (182625, 3)
C.shape = (27, 10)
C[Xtr].shape = (182625, 3, 10)


In [306]:
# Training loop
max_steps = 40000
batch_size = 64
lossi = []


def forward(params, X):

        # forward pass
    emb = params['C'][X]
    embcat = emb.reshape(emb.shape[0], -1)

    hpreact = embcat @ params['W1'] + params['b1']
    #hpreact = bngain*(hpreact - bnmeanr)/bnstdr + bnbias

    h = jnp.tanh(hpreact)
    logits = h @ params['W2'] + params['b2']
    return logits

def softmax_cross_entropy(logits,targets):
   
    # Get Log probabilities 
    lprobs = jax.nn.log_softmax(logits)
    
    # Average loss per target
    loss = -jnp.mean(lprobs[jnp.arange(logits.shape[0]), targets])

    return loss


def get_loss(params, X, Y, regularization=True):

    # Get logits
    logits = forward(params, X)
    
    
    # Get loss
    loss = softmax_cross_entropy(logits, Y) 

    if regularization == True:
        pen = 0.00001
        loss += pen*jnp.sum(params['W1']**2) + pen*jnp.sum(params['W1']**2)

    return loss

    

for i in range(max_steps):

    # minibatch construct
    key, ix_key = jax.random.split(key)
    ix = jax.random.randint(ix_key, (batch_size, ),
                            minval=0, maxval=Xtr.shape[0], dtype=jnp.int32)
    Xb, Yb = Xtr[ix], Ytr[ix]

    loss = get_loss(params, Xb, Yb)

    grads = jax.grad(get_loss)(params, Xb, Yb)

    #loss = -jnp.mean(loss1[jnp.arange(Xb.shape[0], Yb)])

    lr = 0.1 if i < max_steps/2 else 0.01
    params = jax.tree_map(lambda p, g: p - lr * g, params, grads)


    NN = int(max_steps/20)
    lossi.append(loss)
    if i % NN == 0:
        aloss = jnp.mean(jnp.array(lossi[-NN:]))
        print(f'{i:7d}/{max_steps:7d}: loss = {aloss:.4f}')


    #break


      0/  40000: loss = 3.3785
   2000/  40000: loss = 2.4202
   4000/  40000: loss = 2.2946
   6000/  40000: loss = 2.2615
   8000/  40000: loss = 2.2318
  10000/  40000: loss = 2.2174
  12000/  40000: loss = 2.2028
  14000/  40000: loss = 2.1930
  16000/  40000: loss = 2.1895
  18000/  40000: loss = 2.1715
  20000/  40000: loss = 2.1724
  22000/  40000: loss = 2.1141
  24000/  40000: loss = 2.1098
  26000/  40000: loss = 2.1150
  28000/  40000: loss = 2.1200
  30000/  40000: loss = 2.1107
  32000/  40000: loss = 2.1089
  34000/  40000: loss = 2.1081
  36000/  40000: loss = 2.1103
  38000/  40000: loss = 2.1077


In [309]:
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte)
    }[split]

    # forward pass
    emb = params['C'][x]
    embcat = emb.reshape(emb.shape[0], -1)

    hpreact = embcat @ params['W1'] + params['b1']
    #hpreact = bngain*(hpreact - bnmeanr)/bnstdr + bnbias

    h = jnp.tanh(hpreact)
    logits = h @ params['W2'] + params['b2']
    loss = softmax_cross_entropy(logits,y)
    
    print(f'{split:5s} loss = {loss.item():.4f}')


split_loss('train')
split_loss('val')

train loss = 2.0966
val   loss = 2.1314


In [308]:
seed = 42
key = jax.random.PRNGKey(seed)


for _ in range(20):

    out = []
    context = [0] * block_size
    while True:
        
        logits = forward(params, jnp.array([context]))

        key, ix_key = jax.random.split(key)
        ix = jax.random.categorical(ix_key,logits).item()

        # Uniform logits to compare with
        #ix = jax.random.categorical(ix_key,jnp.ones(logits.shape)).item()

        context = context[1:] + [ix]
        out.append(ix)
        
        if ix == 0:
            break
    
    print(''.join(itos[i] for i in out))




koeanleynne.
stel.
arwi.
sanie.
elszin.
kayneya.
bena.
jamyike.
holy.
kaieana.
peanahani.
emahonielilan.
hcce.
eleslynn.
cohiefe.
ghon.
cani.
carrossiyah.
hilah.
chris.
